In [ ]:
"""
Mounting the Google drive
"""
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Root Path

In [1]:
root_path = ''
grade_path = ''

['configure.ini']

## Load Pretrained Model

In [ ]:
# pretrained model
# model.load()
# predictions = model(X_test_PCA_LD)

In [ ]:
import pandas as pd
# Assuming the CSV file has columns 'sample_id' and 'ground_truth'


def calculate_accuracy(ground_truths, predictions):
    if len(ground_truths) != len(predictions):
        msg = "The number of predictions does not match the number of ground truths."
        raise ValueError(msg)
    correct_predictions = 0
    for sample_id, ground_truth in ground_truths.items():
        if predictions.get(sample_id) == ground_truth:
            correct_predictions += 1
    return correct_predictions / len(ground_truths)

def grade_predictions(predictions, grade_path=grade_path):
    df = pd.read_csv(grade_path + "/solution.csv")
    ground_truths = dict(zip(df['filename'], df['ground_truth']))
    accuracy = calculate_accuracy(ground_truths, predictions)
    return accuracy

# Example usage
predictions = {'id1': 'A', 'id2': 'B', 'id3': 'C'}  # Your predictions
accuracy = grade_predictions(predictions)
print(f"Prediction Accuracy: {accuracy * 100}%")

Prediction Accuracy: 100.0%
